In [ ]:
# python -m pip install --user tensorflow-gpu

In [ ]:
# tensorflow_gpu-2.6.0/ python 3.6~3.9 / 컴파일러 MSVC 2019 / 빌드도구 Bazel 3.7.2 / cuDNN 8.1 / CUDA 11.2

In [ ]:
# https://www.doitgrow.com/28

In [159]:
# import pandas as pd

# pd.__version__

'1.3.4'

In [41]:
# from music21 import environment
# us = environment.Environment()    
# us['musescoreDirectPNGPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'

In [42]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9666690890021553202
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5765005312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15022828234931673585
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


## GAN 

In [7]:
import tensorflow as tf
from music21 import*
from __future__ import print_function, division
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import CuDNNLSTM
from music21.exceptions21 import StreamException
from tensorflow.keras.utils import plot_model
import os

In [8]:
environment.set('musescoreDirectPNGPath', 'C:\\Program Files\\MuseScore 3\\bin\\MuseScore3.exe')

In [9]:
def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []
    for file in glob.glob("fantasy/*.mid"):
        try :
            midi = converter.parse(file)
            print("Parsing %s" % file)
            notes_to_parse = None
        except :
            print(file)

        try: 
            s2 = instrument.partitionByInstrument(midi) # 악기 분류
            notes_to_parse = s2.parts[0].recurse() # 2개 이상 악기일 때 노트값
        except: 
            notes_to_parse = midi.flat.notes # 단일 악기일 때 노트값
        # 노트값 중에 단일음과 화음을 분류해 저장해 준다.
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [10]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # 모든 계이름 중복 제거해 sorted로 정렬 
    pitchnames = sorted(set(item for item in notes))

    # 계이름을 숫자로 바꾸는 dictionary를 생성
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # 입력시퀀스를 만들고 그에 따른 출력을 만든다
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # LSTM Layer에 맞게 데이터를 reshape
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # 입력 값을 -1과 1사이로 정규화 
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [11]:
def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # 입력 시퀀스를 랜덤한 값으로 시작
    start = numpy.random.randint(0, len(network_input)-1)
    
    # 중복 제거 및 정렬한 계이름들을 숫자로 매핑하는 dictionary 생성
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []
    # 노트 500개 생성
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [12]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        # 패턴(출력값)이 코드(chord) 일 때
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # 패턴이 노트일 때
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            
        # offset에 음이 쌀이지 않게 각 반복마다 offset 증가 
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='./output/{}.mid'.format(filename))

In [13]:
class GAN():
    def __init__(self, rows):
        # input shape 설정
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        # noise dimension 
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # 판별자 학습 -> 진짜,가짜를 판별하므로 손실함수는 이진분류에 적합한 binary_crossentropy
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        # 생성자 학습
        self.generator = self.build_generator()

        # 생성자는 노이즈를 입력으로 받아 노트 시퀀스를 생성
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # 모델을 합쳐서 학습하기 때문에 discriminator는 따로 학습을 시키지 않음, generator만 학습
        self.discriminator.trainable = False

        # 판별자는 생성된 시퀀스를 입력으로 받아 진짜, 가짜를 판별
        validity = self.discriminator(generated_seq)

        # generator와 discriminator 모델 합침. (노이즈가 인풋으로 들어가서 판별결과가 아웃풋으로 나오게)
        # discriminator를 속이도록 generator를 학습
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        # 판별자 모델
        # return_sequences=True time step별 hidden state들이 출력된다
        # 양방향(Bidirectional)LSTM 모델을 사용해 미래 결과값도 학습해 예측
        # 0~1 사이의 확률값으로 만들기 위해 활성함수 sigmoid 사용
    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dropout(0.5))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)
        
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
        # 생성자 모델
        # 입력 차원을 받아 256개의 뉴런을 가진 레이어로 보내고
        # 레이어 크기를 512, 1024로 증가
        # 마지막에 reshape로 레이어 크기를 줄여줌
        # BatchNormalization을 이용해 배치 단위 별로 데이터가 다양한 분포를 가지더라도 
        # 각 배치별 평균, 분산을 이용해 정규화
        # 활성 함수 LeakyReLU : 출력>0  -> 그대로, 출력<0 -> 정해진 작은 숫자를 곱하는 함수
        # 생성자 마지막 레이어에는 출력값을 픽셀값의 범위인 -1과 1 사이로 만들어주기 위해 Tanh 사용
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        
        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50): # batch_size=128

        # 데이터 로드 및 변환
        notes = get_notes()
        # 노트의 총 가짓수 (중복 제거 위해 set)
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # 진짜는 1, 가짜는 0
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):
            try:
                # 판별자 학습
                # 노트 시퀀스의 배치 사이즈만큼 랜덤으로 뽑음
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                real_seqs = X_train[idx]

                # 샘플 노이즈 생성
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # 노이즈를 생성자에 넣어서 새로운 노트 시퀀스 생성
                gen_seqs = self.generator.predict(noise)

                # 판별자를 학습함
                # 판별자가 판별한 값과 real과 fake값이 각각 같이 들어가서 binary_crossentropy으로 계산되어 업데이트
                d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
                d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
                # real을 넣었을 때와 fake를 넣었을 때의 discriminator의 loss값과 accracy값의 평균을 구함
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


                # 생성자 학습
                # 생성자는 노이즈를 입력받아 생성한 값이 판별자가 진짜라고 식별할 수 있도록 하는게 목적
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                g_loss = self.combined.train_on_batch(noise, real)

                # sample_interval마다 loss와 accracy 프린트
                if epoch % sample_interval == 0:
                    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                    self.disc_loss.append(d_loss[0])
                    self.gen_loss.append(g_loss)
            except:
                pass
#             if epoch%1000 == 0 :
#                 self.generate(notes)
                
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # 계이름을 중복 제거해 정렬한 값을 숫자와 매핑함
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # 랜덤 노이즈를 사용해 시퀀스 생성
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [((x+1)*len(pitchnames))/2 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('./output/GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [16]:
if __name__ == '__main__':
    gan = GAN(rows=100)    
    gan.train(epochs=10000, batch_size=32, sample_interval=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm_4 (CuDNNLSTM)    (None, 100, 512)          1054720   
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             4202496   
 nal)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_14 (Dense)            (None, 512)               524800    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 512)               0         
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                      

57 [D loss: 0.483227, acc.: 79.69%] [G loss: 1.830982]
58 [D loss: 0.462307, acc.: 82.81%] [G loss: 1.985598]
59 [D loss: 0.326352, acc.: 84.38%] [G loss: 2.038188]
60 [D loss: 0.544891, acc.: 78.12%] [G loss: 1.786743]
61 [D loss: 0.413409, acc.: 78.12%] [G loss: 1.815842]
62 [D loss: 0.605957, acc.: 68.75%] [G loss: 1.731289]
63 [D loss: 0.351978, acc.: 85.94%] [G loss: 1.766732]
64 [D loss: 0.518209, acc.: 78.12%] [G loss: 1.672523]
65 [D loss: 0.524149, acc.: 76.56%] [G loss: 1.617331]
66 [D loss: 0.382269, acc.: 82.81%] [G loss: 1.790822]
67 [D loss: 0.380248, acc.: 92.19%] [G loss: 2.313525]
68 [D loss: 0.641609, acc.: 76.56%] [G loss: 1.666957]
69 [D loss: 0.787657, acc.: 59.38%] [G loss: 1.070089]
70 [D loss: 0.575701, acc.: 84.38%] [G loss: 1.162861]
71 [D loss: 0.550945, acc.: 71.88%] [G loss: 1.162320]
72 [D loss: 0.649333, acc.: 62.50%] [G loss: 1.128602]
73 [D loss: 0.649907, acc.: 68.75%] [G loss: 1.135275]
74 [D loss: 0.636994, acc.: 73.44%] [G loss: 1.103733]
75 [D loss

205 [D loss: 0.681915, acc.: 51.56%] [G loss: 0.902714]
206 [D loss: 0.694473, acc.: 54.69%] [G loss: 0.838728]
207 [D loss: 0.719393, acc.: 48.44%] [G loss: 0.724957]
208 [D loss: 0.680624, acc.: 59.38%] [G loss: 0.742732]
209 [D loss: 0.691291, acc.: 57.81%] [G loss: 0.805041]
210 [D loss: 0.694205, acc.: 60.94%] [G loss: 0.830806]
211 [D loss: 0.701327, acc.: 53.12%] [G loss: 0.961118]
212 [D loss: 0.714719, acc.: 53.12%] [G loss: 0.907965]
213 [D loss: 0.710825, acc.: 53.12%] [G loss: 0.874716]
214 [D loss: 0.735694, acc.: 48.44%] [G loss: 0.837745]
215 [D loss: 0.732066, acc.: 45.31%] [G loss: 0.793321]
216 [D loss: 0.688750, acc.: 56.25%] [G loss: 0.804358]
217 [D loss: 0.695863, acc.: 54.69%] [G loss: 0.775461]
218 [D loss: 0.700284, acc.: 46.88%] [G loss: 0.781277]
219 [D loss: 0.680536, acc.: 56.25%] [G loss: 0.787085]
220 [D loss: 0.693411, acc.: 54.69%] [G loss: 0.826944]
221 [D loss: 0.723555, acc.: 50.00%] [G loss: 0.799930]
222 [D loss: 0.673224, acc.: 62.50%] [G loss: 0.

353 [D loss: 0.690428, acc.: 57.81%] [G loss: 0.832582]
354 [D loss: 0.691556, acc.: 53.12%] [G loss: 0.855539]
355 [D loss: 0.670519, acc.: 60.94%] [G loss: 0.892229]
356 [D loss: 0.713056, acc.: 39.06%] [G loss: 0.864617]
357 [D loss: 0.692850, acc.: 60.94%] [G loss: 0.789807]
358 [D loss: 0.666198, acc.: 62.50%] [G loss: 0.846805]
359 [D loss: 0.661559, acc.: 64.06%] [G loss: 0.863640]
360 [D loss: 0.675691, acc.: 57.81%] [G loss: 0.913785]
361 [D loss: 0.696745, acc.: 51.56%] [G loss: 0.831319]
362 [D loss: 0.679229, acc.: 56.25%] [G loss: 0.840208]
363 [D loss: 0.741458, acc.: 40.62%] [G loss: 0.845348]
364 [D loss: 0.690236, acc.: 59.38%] [G loss: 0.861658]
365 [D loss: 0.674036, acc.: 62.50%] [G loss: 0.839178]
366 [D loss: 0.694585, acc.: 54.69%] [G loss: 0.854904]
367 [D loss: 0.659375, acc.: 64.06%] [G loss: 0.812479]
368 [D loss: 0.669057, acc.: 57.81%] [G loss: 0.824517]
369 [D loss: 0.719165, acc.: 50.00%] [G loss: 0.785208]
370 [D loss: 0.697477, acc.: 53.12%] [G loss: 0.

501 [D loss: 0.695781, acc.: 54.69%] [G loss: 0.693557]
502 [D loss: 0.688673, acc.: 53.12%] [G loss: 0.695597]
503 [D loss: 0.693658, acc.: 53.12%] [G loss: 0.695323]
504 [D loss: 0.686454, acc.: 57.81%] [G loss: 0.698955]
505 [D loss: 0.692147, acc.: 54.69%] [G loss: 0.688651]
506 [D loss: 0.698052, acc.: 42.19%] [G loss: 0.678825]
507 [D loss: 0.694366, acc.: 43.75%] [G loss: 0.684887]
508 [D loss: 0.691937, acc.: 54.69%] [G loss: 0.676963]
509 [D loss: 0.700192, acc.: 46.88%] [G loss: 0.700865]
510 [D loss: 0.700335, acc.: 45.31%] [G loss: 0.683044]
511 [D loss: 0.692586, acc.: 54.69%] [G loss: 0.707887]
512 [D loss: 0.690329, acc.: 56.25%] [G loss: 0.694237]
513 [D loss: 0.696480, acc.: 45.31%] [G loss: 0.714653]
514 [D loss: 0.686127, acc.: 62.50%] [G loss: 0.709576]
515 [D loss: 0.695569, acc.: 50.00%] [G loss: 0.708267]
516 [D loss: 0.696512, acc.: 53.12%] [G loss: 0.708145]
517 [D loss: 0.693860, acc.: 54.69%] [G loss: 0.710904]
518 [D loss: 0.693113, acc.: 43.75%] [G loss: 0.

...

9311 [D loss: 0.472389, acc.: 82.81%] [G loss: 1.916754]
9312 [D loss: 0.417285, acc.: 84.38%] [G loss: 1.822439]
9313 [D loss: 0.380576, acc.: 87.50%] [G loss: 1.728435]
9314 [D loss: 0.417621, acc.: 85.94%] [G loss: 1.794545]
9315 [D loss: 0.490689, acc.: 81.25%] [G loss: 1.950361]
9316 [D loss: 0.363780, acc.: 87.50%] [G loss: 1.980813]
9317 [D loss: 1.013668, acc.: 53.12%] [G loss: 1.539418]
9318 [D loss: 0.553187, acc.: 71.88%] [G loss: 1.513571]
9319 [D loss: 0.581314, acc.: 73.44%] [G loss: 1.625477]
9320 [D loss: 0.681440, acc.: 60.94%] [G loss: 1.631849]
9321 [D loss: 0.583997, acc.: 71.88%] [G loss: 1.573223]
9322 [D loss: 0.561354, acc.: 75.00%] [G loss: 1.583274]
9323 [D loss: 0.505454, acc.: 81.25%] [G loss: 1.510526]
9324 [D loss: 0.580659, acc.: 76.56%] [G loss: 1.470719]
9325 [D loss: 0.522646, acc.: 75.00%] [G loss: 1.607145]
9326 [D loss: 0.607538, acc.: 68.75%] [G loss: 1.471878]
9327 [D loss: 0.719086, acc.: 68.75%] [G loss: 1.493371]
9328 [D loss: 0.450286, acc.: 8

9455 [D loss: 0.584558, acc.: 70.31%] [G loss: 1.389240]
9456 [D loss: 0.516258, acc.: 73.44%] [G loss: 1.401510]
9457 [D loss: 0.583488, acc.: 71.88%] [G loss: 1.403499]
9458 [D loss: 0.587760, acc.: 67.19%] [G loss: 1.215442]
9459 [D loss: 0.566166, acc.: 73.44%] [G loss: 1.221123]
9460 [D loss: 0.521320, acc.: 79.69%] [G loss: 1.186630]
9461 [D loss: 0.578114, acc.: 76.56%] [G loss: 1.273063]
9462 [D loss: 0.509924, acc.: 78.12%] [G loss: 1.144795]
9463 [D loss: 0.529865, acc.: 70.31%] [G loss: 1.210225]
9464 [D loss: 0.783317, acc.: 62.50%] [G loss: 1.291336]
9465 [D loss: 0.529742, acc.: 73.44%] [G loss: 1.362419]
9466 [D loss: 0.559352, acc.: 65.62%] [G loss: 1.258854]
9467 [D loss: 0.667565, acc.: 64.06%] [G loss: 1.371693]
9468 [D loss: 0.487521, acc.: 78.12%] [G loss: 1.215981]
9469 [D loss: 0.540535, acc.: 78.12%] [G loss: 1.221797]
9470 [D loss: 0.542090, acc.: 70.31%] [G loss: 1.148927]
9471 [D loss: 0.628546, acc.: 65.62%] [G loss: 1.249789]
9472 [D loss: 0.467636, acc.: 8

9599 [D loss: 1.528209, acc.: 46.88%] [G loss: 1.650747]
9600 [D loss: 0.507897, acc.: 79.69%] [G loss: 1.472013]
9601 [D loss: 0.644066, acc.: 65.62%] [G loss: 1.253459]
9602 [D loss: 0.632661, acc.: 71.88%] [G loss: 1.122983]
9603 [D loss: 0.542733, acc.: 79.69%] [G loss: 1.076016]
9604 [D loss: 0.619594, acc.: 64.06%] [G loss: 1.153075]
9605 [D loss: 0.589405, acc.: 68.75%] [G loss: 1.058333]
9606 [D loss: 0.535649, acc.: 81.25%] [G loss: 1.064896]
9607 [D loss: 0.551437, acc.: 71.88%] [G loss: 1.195508]
9608 [D loss: 0.427266, acc.: 79.69%] [G loss: 1.304280]
9609 [D loss: 0.437197, acc.: 81.25%] [G loss: 1.397843]
9610 [D loss: 0.466446, acc.: 81.25%] [G loss: 1.278517]
9611 [D loss: 0.579413, acc.: 71.88%] [G loss: 1.390559]
9612 [D loss: 0.489584, acc.: 78.12%] [G loss: 1.313393]
9613 [D loss: 0.430295, acc.: 81.25%] [G loss: 1.346859]
9614 [D loss: 0.379638, acc.: 79.69%] [G loss: 1.457628]
9615 [D loss: 0.390381, acc.: 85.94%] [G loss: 1.384024]
9616 [D loss: 0.410583, acc.: 8

9744 [D loss: 0.462375, acc.: 79.69%] [G loss: 1.649057]
9745 [D loss: 0.374747, acc.: 87.50%] [G loss: 1.778951]
9746 [D loss: 0.465410, acc.: 82.81%] [G loss: 1.650878]
9747 [D loss: 0.459271, acc.: 78.12%] [G loss: 1.817840]
9748 [D loss: 0.826959, acc.: 64.06%] [G loss: 1.156187]
9749 [D loss: 0.542888, acc.: 67.19%] [G loss: 1.244835]
9750 [D loss: 0.593235, acc.: 67.19%] [G loss: 1.123728]
9751 [D loss: 0.638198, acc.: 62.50%] [G loss: 1.363241]
9752 [D loss: 0.524230, acc.: 75.00%] [G loss: 1.304858]
9753 [D loss: 0.555668, acc.: 68.75%] [G loss: 1.326332]
9754 [D loss: 0.615601, acc.: 68.75%] [G loss: 1.018594]
9755 [D loss: 0.700388, acc.: 54.69%] [G loss: 1.267932]
9756 [D loss: 0.404095, acc.: 87.50%] [G loss: 1.352341]
9757 [D loss: 0.592124, acc.: 70.31%] [G loss: 1.603205]
9758 [D loss: 0.757120, acc.: 62.50%] [G loss: 1.681806]
9759 [D loss: 0.625993, acc.: 70.31%] [G loss: 1.452497]
9760 [D loss: 0.750815, acc.: 57.81%] [G loss: 1.305198]
9761 [D loss: 0.540131, acc.: 6

9888 [D loss: 0.286950, acc.: 90.62%] [G loss: 1.976373]
9889 [D loss: 0.268307, acc.: 90.62%] [G loss: 1.957526]
9890 [D loss: 0.427142, acc.: 82.81%] [G loss: 2.353566]
9891 [D loss: 0.486681, acc.: 76.56%] [G loss: 2.266272]
9892 [D loss: 0.350961, acc.: 85.94%] [G loss: 2.094621]
9893 [D loss: 0.380213, acc.: 82.81%] [G loss: 1.810000]
9894 [D loss: 0.224716, acc.: 93.75%] [G loss: 2.222946]
9895 [D loss: 0.181421, acc.: 93.75%] [G loss: 2.495610]
9896 [D loss: 0.251680, acc.: 87.50%] [G loss: 2.404298]
9897 [D loss: 0.183552, acc.: 95.31%] [G loss: 2.465201]
9898 [D loss: 0.541722, acc.: 73.44%] [G loss: 2.630353]
9899 [D loss: 0.452353, acc.: 84.38%] [G loss: 2.164311]
9900 [D loss: 0.347141, acc.: 84.38%] [G loss: 2.132649]
9901 [D loss: 0.263775, acc.: 87.50%] [G loss: 2.041020]
9902 [D loss: 0.431377, acc.: 79.69%] [G loss: 1.934329]
9903 [D loss: 0.164112, acc.: 93.75%] [G loss: 2.078686]
9904 [D loss: 0.203393, acc.: 93.75%] [G loss: 2.305432]
9905 [D loss: 0.302633, acc.: 8